<a href="https://colab.research.google.com/github/a-mhamdi/jlai/blob/main/Codes/Julia/Part-3/cnn/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HANDWRITTEN DIGITS RECOGNITION USING CNN
---

This notebook demonstrates handwritten digit classification using a **Convolutional Neural Network** (**CNN**). The implementation utilizes the `Flux.jl` library in Julia, with `CUDA.jl` for **GPU** acceleration.

## Project Setup

In [ ]:
versioninfo()

In [ ]:
# ;wget -O Project.toml https://raw.githubusercontent.com/a-mhamdi/jlai/refs/heads/main/Codes/Julia/Part-3/cnn/Project.toml

In [ ]:
# ;sed -i -e '/^Pl[uo]/d' -e '/^Image/d' Project.toml

We write the package dependencies to a `Project.toml` file. This file is used by Julia's package manager `Pkg` to specify the exact versions of the packages needed here.

In [ ]:
pkgs = """[deps]
BSON = "fbb218c0-5317-5bc6-957e-2ee96dd4b1f0"
CUDA = "052768ef-5323-5732-b1bb-66c8b64840ba"
CairoMakie = "13f3f980-e62b-5c42-98c6-ff1f3baf88f0"
Flux = "587475ba-b771-5e3f-ad9e-33799f191a9c"
ImageInTerminal = "d8c32880-2388-543b-8c61-d9f865259254"
ImageShow = "4e3cecfd-b093-5904-9786-8bbb286a6a31"
JLD2 = "033835bb-8acc-5ee8-8aae-3f567f8a3819"
MLDatasets = "eb30cadb-4394-5ae3-aed4-317e484a6458"
Markdown = "d6f4376e-aef5-505a-96c1-9c027394607a"
ProgressMeter = "92933f4c-e287-5a05-a399-4b506db050ca"
Statistics = "10745b16-79ce-11e8-11f9-7d13ad32a3b2"
cuDNN = "02a925ec-e4fe-4b08-9a7e-0d78e3d38ccd"
"""

open("Project.toml", "w") do file
    write(file, pkgs)
end

Activate the project environment and instantiates the packages listed in the `Project.toml` file.

In [ ]:
_ = begin
  import Pkg;
  Pkg.activate(".");
  Pkg.instantiate();
end

Status of the installed packages in the current environment, showing their versions and any compatibility issues.

In [ ]:
Pkg.status()

Import the `Flux` library, which is essential for building and training the neural network.

In [ ]:
using Flux
using Flux: DataLoader
using Flux: onecold, onehotbatch

Import the `CUDA` package and displays information about the `CUDA` version and available **GPU** devices.

In [ ]:
using CUDA
CUDA.versioninfo()

## Load and process data

In [ ]:
Base.@kwdef mutable struct HyperParams
    η = 3f-3                # Learning rate
    batchsize = 64          # Batch size
    epochs = 8              # Number of epochs
    split = :train          # Split data into `train` and `test`
end

Load the **MNIST** dataset

In [ ]:
using MLDatasets

In [ ]:
d = MNIST()

The function `get_data` loads the MNIST dataset, normalizes the features by dividing by 255, reshapes the data to include a channel dimension, and one-hot encodes the labels. It then creates a `DataLoader` for efficient batching and moves the data to the GPU.

In [ ]:
function get_data(; kws...)
    args = HyperParams(; kws...);
    # Split and normalize data
    data = MNIST(split=args.split);
    X, y = data.features ./ 255, data.targets;
    X = reshape(X, (28, 28, 1, :));
    y = onehotbatch(y, 0:9);
    loader = DataLoader((X, y); batchsize=args.batchsize, shuffle=true) |> gpu;
    return loader
end

In [ ]:
train_loader = get_data();
test_loader = get_data(split=:test);

Check the type of the first batch of data from the `get_data` function. The output will indicate whether the data is stored as an `Array` (on the CPU) or a `CuArray` (on the GPU).

In [ ]:
x1, y1 = first(get_data());
# CPU => Array...
# GPU => CuArray...
typeof((x1, y1))

Transform sample training data to an image. View the image and check the corresponding digit value.

In [ ]:
using Statistics

In [ ]:
idx = rand(1:6_000, 3)

In [ ]:
using ImageInTerminal, ImageShow

In [ ]:
convert2image(d, idx)

In [ ]:
"Digits are $(d.targets[idx])"

## Model Architecture Design

Let us readjust and use the original `LeNet-5` Architecture.

In [ ]:
lenet5 = Chain(
    # Layer C1: Convolutional layer - 6 feature maps, 5x5 kernel
    Conv((5, 5), 1 => 6, relu),

    # Layer S2: Subsampling (Max Pooling) - 2x2
    MaxPool((2, 2)),

    # Layer C3: Convolutional layer - 16 feature maps, 5x5 kernel
    Conv((5, 5), 6 => 16, relu),

    # Layer S4: Subsampling (Max Pooling) - 2x2
    MaxPool((2, 2)),

    # Flatten for fully connected layers
    Flux.flatten,

    # Layer F5: Fully connected - 120 units
    Dense(256, 120, tanh),

    # Layer F6: Fully connected - 84 units
    Dense(120, 84, tanh),

    # Output layer: 10 classes (for digits 0-9)
    Dense(84, 10),
) |> gpu

The input `X` is a batch of images with dimensions `(width=28, height=28, channels=1, batchsize)`.

In [ ]:
lenet5(x1) |> out -> softmax(out; dims=1)

## Training and Validation

In [ ]:
using ProgressMeter: Progress, next!

In [ ]:
function train(; kws...)
    args = HyperParams(; kws...)
    # Define the loss function
    l(α, β) = Flux.logitcrossentropy(α, β)
    # Define the accuracy metric
    acc(α, β) = mean(onecold(α) .== onecold(β))
    # Optimizer
    optim_state = Flux.setup(Adam(args.η), lenet5);

    vec_loss = []
    vec_acc = []

    for epoch in 1:args.epochs
        printstyled("\t***\t === EPOCH $(epoch) === \t*** \n", color=:magenta, bold=true)
        @info "TRAINING"
        prg_train = Progress(length(train_loader))
        for (X, y) in train_loader
            loss, grads = Flux.withgradient(lenet5) do m
                ŷ = m(X);
                l(ŷ, y);
            end
            Flux.update!(optim_state, lenet5, grads[1]); # Upd `W` and `b`
            # Show progress meter
            next!(prg_train, showvalues=[(:loss, loss)])
        end
        @info "TESTING"
        prg_test = Progress(length(test_loader))
        for (X, y) in test_loader
            ŷ = lenet5(X);
            push!(vec_loss, l(ŷ, y));  # log `loss` value -> `vec_loss` vector
            push!(vec_acc, acc(ŷ, y)); # log `accuracy` value -> `vec_acc` vector
          	# Show progress meter
            next!(prg_test, showvalues=[(:loss, vec_loss[end]), (:accuracy, vec_acc[end])])
        end
    end
    vec_loss, vec_acc
end

In [ ]:
vec_loss, vec_acc = train();

In [ ]:
using CairoMakie

fig = Figure(size = (800, 800))
ax1 = Axis(
    fig[1, 1],
    xlabel = "Iteration",
    ylabel = "Loss",
    title = "Test Loss vs. Iteration"
)
ax2 = Axis(
    fig[2, 1],
    xlabel = "Iteration",
    ylabel = "Accuracy",
    title = "Test Accuracy vs. Iteration"
)

# Scatter plot the `vec_loss`
scatter!(ax1, vec_loss[1:10:end], color=:red, markersize=5)
ylims!(ax1, 0, 2)

# Scatter plot the `acc_loss`
scatter!(ax2, vec_acc[1:10:end], color=:blue, markersize=5)
ylims!(ax2, 0, 1)

# Display the figure
fig

Sampling from `test_loader` and comparison between predicted and actual outputs:

In [ ]:
xs, ys = first(test_loader)
yp = xs |> lenet5 |> softmax |> out -> onecold(out, 0:9) |> cpu
ys = onecold(ys) .- 1 |> cpu;

In [ ]:
idx = rand(1:HyperParams().batchsize, 8)
for i ∈ idx
    @info "**Prediction is $(yp[i]). Label is $(ys[i]).**"
end

## Save and Load

In [ ]:
cnn_model = lenet5 |> cpu;

In [ ]:
folder = "models"
isdir(folder) || mkdir(folder)

In [ ]:
using Serialization

filename_1 = joinpath(folder, "cnn.jls")
serialize(filename_1, cnn_model)

In [ ]:
using BSON

filename_3 = joinpath(folder, "cnn.bson")
BSON.@save filename_3 bson_mdl=cnn_model

In [ ]:
using JLD2

filename_2 = joinpath(folder, "cnn.jld2")
JLD2.@save filename_2 jld2_mdl=cnn_model

In [ ]:
# cnn_model = nothing

In [ ]:
idx = rand(1:6000)
sample = d.features[:, :, idx] |> im -> reshape(im, (28, 28, 1, 1))
convert2image(d, idx)

In [ ]:
stream_mdl = deserialize(filename_1)
pred = stream_mdl(sample) |> out -> (onecold(out) .- 1)[1]
print(d.targets[idx], ' ', pred)

In [ ]:
BSON.@load filename_3 bson_mdl
pred = bson_mdl(sample) |> out -> (onecold(out) .- 1)[1]
print(d.targets[idx], ' ', pred)

In [ ]:
JLD2.@load filename_2 jld2_mdl
pred = jld2_mdl(sample) |> out -> (onecold(out) .- 1)[1]
print(d.targets[idx], ' ', pred)